In [4]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from cleantext import clean

import swifter

In [5]:
def preprocess_row(row, date, parent, headline):
    text = row['text']
    text = clean(text, lower=False, no_urls=True, no_emails=True, zero_digits=True)
    tokens = []
    
    if date:
        date = datetime.fromtimestamp(row['timestamp'])
        weekday = ' xxweekday_' + str(date.weekday())
        hour = ' xxhour_' + str(date.hour // 3) # reduce option space
        tokens += [weekday, hour]
    if parent:
        is_reply = ' xxreplysep xxreply_' + str(pd.isnull(row['parentid'])).lower()
        tokens.append(is_reply)
    if headline:
        headline = clean(row['headline'], lower=False, no_urls=True, no_emails=True, zero_digits=True)
        tokens.append(headline + '.')
        
    return ' '.join([text] + tokens)

In [6]:
def preprocess(IN_PATH, OUT_PATH, test=False, date=True, parent=True, headline=True):
    out_base = f"dat_{date}_par_{parent}_hea_{headline}".lower()
    files = ['train.csv', 'val.csv', 'test.csv'] if test else ['train.csv', 'val.csv']
    for f in files:
        df = pd.read_csv(IN_PATH/f)
        df['text_proc'] = df.swifter.apply(lambda x: preprocess_row(x, date=date, parent=parent,headline=headline), axis=1)
        ! mkdir -p {OUT_PATH/out_base}
        df.to_csv(OUT_PATH/out_base/f)

In [7]:
hea = False
dat = False
par = True

In [8]:
IN = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/split/')
OUT = Path('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/cls/')
preprocess(IN, OUT, headline=hea, date=dat, parent=par, test=True)

Pandas Apply: 100%|██████████| 553/553 [00:00<00:00, 962.48it/s]


In [9]:
IN = Path('~/data/ynacc_proc/replicate/lmdata_complete')
OUT = Path('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/')
preprocess(IN, OUT, headline=hea, date=dat, parent=par)

Pandas Apply: 100%|██████████| 38512/38512 [00:35<00:00, 1092.52it/s]


In [3]:
df = []
for x in ['train.csv', 'val.csv']:
    df.append(pd.read_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false/' + x))
df = pd.concat(df)

In [4]:
res = df.groupby('sdid').apply(lambda x: x.sort_values(by=['commentindex']))

In [5]:
res

Unnamed: 0             commentagreement  \
sdid                                                     
4      161346      161346                          NaN   
       161350      161350                          NaN   
       161363      161363                          NaN   
       168315      168315                          NaN   
       172477      172477                          NaN   
       178250      178250                          NaN   
9      23366        23366                          NaN   
       23883        23883                          NaN   
       24903        24903                          NaN   
21     45320        45320                          NaN   
       45426        45426                          NaN   
       45446        45446                          NaN   
       45458        45458                          NaN   
       45475        45475                          NaN   
       45543        45543                          NaN   
       45554        45554                          NaN   
24     43575        43575                          NaN   
       43663        43663                          NaN   
       44090        44090                          NaN   
       44129        44129                          NaN   
       44850        44850                          NaN   
       46838        46838                          NaN   
       51137        51137                          NaN   
27     50792        50792                          NaN   
       50793        50793                          NaN   
       51106        51106  Disagreement with commenter   
       51107        51107                          NaN   
       51182        51182                          NaN   
       51183        51183  Disagreement with commenter   
       51306        51306                          NaN   
...                   ...                          ...   
148009 110195      110195                          NaN   
       110228      110228                          NaN   
       110244      110244                          NaN   
       110275      110275                          NaN   
       110290      110290                          NaN   
       110341      110341                          NaN   
       110343      110343                          NaN   
       111056      111056                          NaN   
148010 110169      110169                          NaN   
       110255      110255                          NaN   
       110308      110308                          NaN   
       110317      110317                          NaN   
       110497      110497                          NaN   
148024 162202      162202                          NaN   
       162764      162764                          NaN   
       163005      163005                          NaN   
       163101      163101                          NaN   
       163473      163473                          NaN   
148026 159148      159148                          NaN   
       159507      159507                          NaN   
       159608      159608                          NaN   
       159803      159803                          NaN   
       159986      159986                          NaN   
       160503      160503                          NaN   
       161050      161050                          NaN   
148034 188649      188649                          NaN   
       189353      189353                          NaN   
       195191      195191                          NaN   
       199824      199824                          NaN   
       22128        22128                          NaN   

                                                       commentid  \
sdid                                                               
4      161346  1461615323534-9421e747-ccaa-4d2b-8819-c3c660d6...   
       161350  00002b000000000000000000000000-e944c5ab-1d67-4...   
       161363  00003b000000000000000000000000-3d0d39e9-7397-4...   
       168315  00004

In [41]:
res2 = res.groupby('sdid').apply(lambda x: x['text_proc'].str.cat(sep=' '))

xx = pd.DataFrame(res2).reset_index()

xx['text_proc'] = xx[0]

xx = xx[['sdid', 'text_proc']]

! mkdir -p '/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid/'

xx[:30000].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid/train.csv', index=False)

xx[30000:].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid/val.csv', index=False)

/home/group7/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 'sdid' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [7]:
res2 = res.groupby('sdid').apply(lambda x: ' xx_thread_start xx_comment_start ' + x['text_proc'].str.cat(sep=' xx_comment_end xx_comment_start ') + ' xx_comment_end xx_thread_end ')

xx = pd.DataFrame(res2).reset_index()

xx['text_proc'] = xx[0]

xx = xx[['sdid', 'text_proc']]

! mkdir -p '/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid_sep/'

xx[:30000].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid_sep/train.csv', index=False)

xx[30000:].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/dat_false_par_false_hea_false_bysdid_sep/val.csv', index=False)

/home/group7/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 'sdid' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [6]:
res2 = res.groupby('sdid').apply(lambda x: x['text_proc'].str.cat(sep=' xxbos '))

xx = pd.DataFrame(res2).reset_index()

xx['text_proc'] = xx[0]

xx = xx[['sdid', 'text_proc']]

! mkdir -p '/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/sep_bos/'

xx[:30000].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/sep_bos/train.csv', index=False)

xx[30000:].to_csv('/mnt/data/group07/johannes/ynacc_proc/proper_baseline/lm/sep_bos/val.csv', index=False)

/home/group7/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 'sdid' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
